## Beam Column Strength by 13.8.2 - class 1 and 2 I-shaped sections
No lateral loads applied to column.  Analysis for forces must include $P-\Delta$ effects.

In [1]:
from utils import show
from math import sqrt, pi

In [2]:
from Designer import sst
SST = sst.SST()

In [3]:
Fy = 345.     # ASTM A992
E = 200000.
G = 77000.
phi = 0.9

## Example BC40
### 'Lecture' example of 2020-03-26 Part-3 (no real lectures because of COVID-19)

In [4]:
Dsg = 'W250x101'
Lcol = 5500. # total length of column, in mm
Lx = Lcol       # unbraced length, x-axis
Ly = Lcol       # unbraced length, y-axis
Kx = 1.      # effective length factor
Ky = 1.
Lc = Ly      # unbraced length of compression flange
Cf = 1000.   # factored axial load, kN
Mfx1 = -200.  # moment about strong axis, top and bottom
Mfx2 = 240.  # same sign for double curvature
Mfy1 = 0.    # momemt about weak axis, top and bottom
Mfy2 = 0.
bracedx = True   # braced for bending about strong axis?
bracedy = True   # braced for bending about weak axis

In [5]:
A,D,T,B,W,Ix,Iy,Zx,Zy,Rx,Ry,J,Cw = SST.section(Dsg,'A,D,T,B,W,Ix,Iy,Zx,Zy,Rx,Ry,J,Cw')
show('Dsg,A,D,T,B,W,Rx,Ry,*1E6,Ix,Iy,*1E3,Zx,Zy,J,*1E9,Cw')

Dsg = W250x101   
A   = 12900      
D   = 264        
T   = 19.6       
B   = 257        
W   = 11.9       
Rx  = 113        
Ry  = 65.6       
Ix  = 164 * 1E6  
Iy  = 55.5 * 1E6 
Zx  = 1400 * 1E3 
Zy  = 656 * 1E3  
J   = 1490 * 1E3 
Cw  = 829 * 1E9  


#### Section class, local buckling

In [6]:
fclass = 4
if B/(2.*T) <= 145./sqrt(Fy):
    fclass = 1
elif B/(2.*T) <= 170./sqrt(Fy):
    fclass = 2
elif B/(2.*T) <= 200./sqrt(Fy):
    fclass = 3
    
H = (D - 2.*T)
phiCy = phi*A*Fy * 1E-3
wclass = 4
if (H/W) <= 1100.*(1-0.39*Cf/phiCy)/sqrt(Fy):
    wclass = 1
elif (H/W) <= 1700.*(1-0.61*Cf/phiCy)/sqrt(Fy):
    wclass = 2
elif (H/W) <= 1900.*(1-0.65*Cf/phiCy)/sqrt(Fy):
    wclass = 3
    
sclass = max(fclass,wclass)
show('B/(2.*T),145./sqrt(Fy),170./sqrt(Fy),200./sqrt(Fy),fclass',
     'H,H/W,Cf/phiCy',
     '1100.*(1-0.39*Cf/phiCy)/sqrt(Fy),1700.*(1-0.61*Cf/phiCy)/sqrt(Fy),1900.*(1-0.65*Cf/phiCy)/sqrt(Fy)'
     ,'wclass,sclass,sclass <= 2')
assert sclass <= 2, 'This only works for class 1 or class 2 sections'

B/(2.*T)                         = 6.556  
145./sqrt(Fy)                    = 7.807  
170./sqrt(Fy)                    = 9.152  
200./sqrt(Fy)                    = 10.77  
fclass                           = 1      
H                                = 224.8  
H/W                              = 18.89  
Cf/phiCy                         = 0.2497 
1100.*(1-0.39*Cf/phiCy)/sqrt(Fy) = 53.46  
1700.*(1-0.61*Cf/phiCy)/sqrt(Fy) = 77.59  
1900.*(1-0.65*Cf/phiCy)/sqrt(Fy) = 85.69  
wclass                           = 1      
sclass                           = 1      
sclass <= 2                      = True   


### 13.8.2 (a) -- cross-sectional strength:

In [7]:
Mfx = max(abs(Mfx1),abs(Mfx2))
Mfy = max(abs(Mfy1),abs(Mfy2))
show('Mfx,Mfy')

Mfx = 240 
Mfy = 0   


In [8]:
def kappa(Mf1,Mf2):    
    Mfmax = max(abs(Mf1),abs(Mf2))
    if Mfmax > 0.:
        kap = min(abs(Mf1),abs(Mf2))/Mfmax
        if Mf1*Mf2 > 0:
            return kap
        else:
            return -kap
    return 0.
def U1(L,I,Cf,Mf1,Mf2):
    w1 = max(0.6-0.4*kappa(Mf1,Mf2),0.4)
    Ce = pi*pi*E*I/(L*L) * 1E-3
    return w1,Ce,w1/(1.-Cf/Ce)

In [9]:
Mfx1,Mfx2,kappa(Mfx1,Mfx2),U1(Lx,Ix,Cf,Mfx1,Mfx2)

(-200.0,
 240.0,
 -0.8333333333333334,
 (0.9333333333333333, 10701.587582007634, 1.029537519034267))

In [10]:
if bracedx or bracedy:
    Cr = phi*A*Fy * 1E-3
    Mrx = phi*Zx*Fy * 1E-6
    w1,Ce,U1xc = U1(Lx,Ix,Cf,Mfx1,Mfx2)
    U1x = max(1.0,U1xc)    
    lhs_a = Cf/Cr + 0.85*U1x*Mfx/Mrx
    show('Cf,Cr,w1,Ce,U1xc,U1x,Mfx,Mrx,Cf/Cr,0.85*U1x*Mfx/Mrx')
    if Mfy > 0.:
        Mry = phi*Zy*Fy * 1E-6
        Beta = 0.6
        w1y,Cey,U1yc = U1(Ly,Iy,Cf,Mfy1,Mfy2)
        U1y = max(1.0,U1yc)
        lhs_a += Beta*U1y*Mfy/Mry
        show('Beta,w1y,Cey,U1yc,U1y,Mfy,Mry,Beta*U1y*Mfy/Mry')
else:
    print('Unbraced frame: cross-sectional strength not checked')
    lhs_a = 0.
show('lhs_a,lhs_a <= 1')

Cf               = 1000   
Cr               = 4005   
w1               = 0.9333 
Ce               = 10700  
U1xc             = 1.03   
U1x              = 1.03   
Mfx              = 240    
Mrx              = 434.7  
Cf/Cr            = 0.2497 
0.85*U1x*Mfx/Mrx = 0.4832 
lhs_a      = 0.7328 
lhs_a <= 1 = True   


### 13.8.2 (b) -- overall member strength

In [11]:
assert Mfx > 0. or Mfy > 0., "No end bending moments in either direction"
K = 1.
if Mfx > 0. and Mfy == 0.:
    KLr = K*Lx/Rx
elif Mfx == 0. and Mfy > 0.:
    KLr = K*Ly/Ry
else:
    KLr = K*Ly/Ry
# Cr by 13.3
Fe = pi*pi*E/(KLr*KLr)
lamda = sqrt(Fy/Fe)
n = 1.34
Cr = phi*A*Fy*(1.+lamda**(2*n))**(-1./n) * 1E-3
Mrx = phi*Zx*Fy * 1E-6
U1x = U1(Lx,Ix,Cf,Mfx1,Mfx2)[2] if bracedx else 1.0
lhs_b = Cf/Cr + 0.85*U1x*Mfx/Mrx
show('Cf,K,K*Lx/Rx,K*Ly/Ry,KLr,Fe,lamda,n,Cr,U1x,Mfx,Mrx,Cf/Cr,0.85*U1x*Mfx/Mrx')
if Mfy > 0.:
    Mry = phi*Zy*Fy * 1E-6
    KLry = Ky*Ly/Ry
    Fey = pi*pi*E/(KLry*KLry)
    lamday = sqrt(Fy/Fey)
    Beta = min(0.85,0.6 + 0.4*lamday)
    U1y = U1(Ly,Iy,Cf,Mfy1,Mfy2)[2] if bracedy else 1.0
    lhs_b += Beta*U1y*Mfy/Mry
    show('lamday,Beta,U1y,Mfy,Mry,Beta*U1y*Mfy/Mry')
show('lhs_b,lhs_b <= 1')

Cf               = 1000   
K                = 1      
K*Lx/Rx          = 48.67  
K*Ly/Ry          = 83.84  
KLr              = 48.67  
Fe               = 833.2  
lamda            = 0.6435 
n                = 1.34   
Cr               = 3280   
U1x              = 1.03   
Mfx              = 240    
Mrx              = 434.7  
Cf/Cr            = 0.3048 
0.85*U1x*Mfx/Mrx = 0.4832 
lhs_b      = 0.788 
lhs_b <= 1 = True  


### 13.8.2 (c) lateral torsional buckling strength

In [12]:
KLr = max(Kx*Lx/Rx,Ky*Ly/Ry)
Fe = pi*pi*E/(KLr**2)
lamda = sqrt(Fy/Fe)
n = 1.34
Cr = phi*A*Fy*(1.+lamda**(2*n))**(-1./n) * 1E-3
show('Cf,Kx,Lx,Rx,Ky,Ly,Ry,Kx*Lx/Rx,Ky*Ly/Ry,KLr,Fe,lamda,n,Cr')
L = Lc
P = E*Iy*G*J
Q = Iy*Cw*(pi*E/L)**2
kap = kappa(Mfx1,Mfx2)
omega2 = min(1.75 + 1.05*kap + 0.3*kap**2, 2.5)
Mu = (omega2*pi/L) * sqrt(P+Q) * 1E-6
Mp = Zx*Fy * 1E-6
if Mu <= 0.67*Mp:
    Mrx = phi*Mu
else:
    Mrx = min(1.15*phi*Mp*(1-0.28*Mp/Mu),phi*Mp)
U1x = max(U1(Lx,Ix,Cf,Mfx1,Mfx2)[2],1.0) if bracedx else 1.0
show('L,kap,omega2,Mu,Mp,Mfx,U1x,Mrx,Cf/Cr,0.85*U1x*Mfx/Mrx')
lhs_c = Cf/Cr + 0.85*U1x*Mfx/Mrx
if Mfy > 0.:
    Mry = phi*Zy*Fy * 1E-6
    KLry = Ky*Ly/Ry
    Fey = pi*pi*E/(KLry*KLry)
    lamday = sqrt(Fy/Fey)
    Beta = min(0.85,0.6 + 0.4*lamday)
    U1y = U1(Ly,Iy,Cf,Mfy1,Mfy2)[2] if bracedy else 1.0
    lhs_c += Beta*U1y*Mfy/Mry
    show('Mfy,lamday,Beta,U1y,Mry,Beta*U1y*Mfy/Mry')
show('lhs_c,lhs_c <= 1')

Cf       = 1000  
Kx       = 1     
Lx       = 5500  
Rx       = 113   
Ky       = 1     
Ly       = 5500  
Ry       = 65.6  
Kx*Lx/Rx = 48.67 
Ky*Ly/Ry = 83.84 
KLr      = 83.84 
Fe       = 280.8 
lamda    = 1.108 
n        = 1.34  
Cr       = 2139  
L                = 5500    
kap              = -0.8333 
omega2           = 1.083   
Mu               = 847.1   
Mp               = 483     
Mfx              = 240     
U1x              = 1.03    
Mrx              = 420.1   
Cf/Cr            = 0.4675  
0.85*U1x*Mfx/Mrx = 0.4999  
lhs_c      = 0.9674 
lhs_c <= 1 = True   


In [13]:
lhs_d = Mfx/Mrx
if Mfy > 0.:
    lhs_d += Mfy/Mry
show('lhs_d, lhs_d < 1')

lhs_d     = 0.5713 
lhs_d < 1 = True   


In [14]:
if lhs_a <= 1 and lhs_b <= 1 and lhs_c < 1 and lhs_d < 1:
    print(Dsg, 'is OK!')
else:
    print(Dsg, 'is NG!')

W250x101 is OK!
